In [9]:
import requests
import time
import pandas as pd
import json
from bs4 import BeautifulSoup
import shutil

In [10]:
master_list = []

In [ ]:
def get_photo(url, pid):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(fr'./imgs/{pid}.jpg', 'wb') as f:
            for chunk in r:
                f.write(chunk)

In [12]:
def get_response(page):
    cookies = {
        'idutm': '13352',
        'aff': '630',
        'v_aff': 'E',
        'G_ENABLED_IDPS': 'google',
        'sp_connu': '21027312%238f7392aecc8cde7f409aef76b4884cec',
        '__stripe_mid': '6e91cbeb-26ad-4ad2-9e8d-8ea430eb1d79c81c8c',
        'PHPSESSID': '689245kq8hbg00firlklkha9pu',
    }

    headers = {
        'Accept': '*/*',
        'Accept-Language': 'es-419,es;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        # Requests sorts cookies= alphabetically
        # 'Cookie': 'idutm=13352; aff=630; v_aff=E; G_ENABLED_IDPS=google; sp_connu=21027312%238f7392aecc8cde7f409aef76b4884cec; __stripe_mid=6e91cbeb-26ad-4ad2-9e8d-8ea430eb1d79c81c8c; PHPSESSID=689245kq8hbg00firlklkha9pu',
        'Referer': 'https://www.superprof.mx/s/todas-materias,M%C3%A9xico,,,1.html?s=4',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-GPC': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
    }

    params = {
        'type': '4',
        'adress': 'México',
        'matiere': 'Todas Materias',
        'place[]': [
            'at_my_place',
            'i_move',
        ],
        'distance': '50',
        'webcam_from': 'in_pays',
        'order_by': 'pertinence_DESC',
        'price': 'all',
        'first_lesson': 'all',
        'response_time': 'all',
        'is_mobile': '2',
        'pmin': '1',
        'pmax': '4401',
        'isV': '1',
        'url': 'https://www.superprof.mx/s/todas-materias,M%C3%A9xico,,,1.html?s=4',
        'page': f'{page}',
    }

    # response = requests.get('https://www.superprof.mx/a/search/', params=params, cookies=cookies, headers=headers)
    response = requests.get('https://www.superprof.mx/a/search/', params=params, headers=headers)
    return response.text

In [ ]:
# pages -> 55029
# "nbPagesTotal": 33019

for i in range (1, 33019):
    
    data_json = json.loads(get_response(i))
    # print(json.dumps(data_json, indent=4))
    results = data_json['mainResults']

    for data in results:
        teacher = {}

        teacher['id'] = data['id_annonce']
        teacher['teacherName'] = data['teacherName']
        teacher['price'] = data['price']
        teacher['_type'] = data['type']

        try:
            teacher['teacherPhoto'] = get_photo(data['teacherPhoto'], teacher['id'])
        except:
            teacher['teacherPhoto'] = None

        try:
            teacher['teacherRating_average'] = data['teacherRating']['average']
        except:
            teacher['teacherRating_average'] = None

        try:
            teacher['teacherRating_count'] = data['teacherRating']['count']
        except:
            teacher['teacherRating_count'] = None

        teacher['url'] = data['url']
        teacher['url_mer'] = data['url_mer']
        teacher['title'] = data['title']
        teacher['teacherCity'] = data['teacherCity']
        teacher['distance'] = data['distance']
        teacher['firstHourFree'] = data['firstHourFree']
        teacher['firstFreeDuration'] = data['firstFreeDuration']
        teacher['is_feminin'] = data['is_feminin']
        teacher['id_membre_statut'] = data['id_membre_statut']
        try:
            teacher['badge'] = data['badge']
        except:
            teacher['badge'] = None
        teacher['senior'] = data['senior']
        teacher['verified'] = data['verified']
        teacher['faceToFace'] = data['faceToFace']
        teacher['webcam'] = data['webcam']
        teacher['responseTime'] = data['responseTime']
        teacher['responseTimeDesc'] = data['responseTimeDesc']
        teacher['key_admin'] = data['key_admin']

#         print(teacher)
        master_list.append(teacher)
#         id_annonce, teacherName, price, _type, teacherPhoto, teacherRating, teacherRating_average, teacherRating_count, teacherRating_label, url, url_mer, title, teacherCity, distance, firstHourFree, firstFreeDuration, is_feminin, id_membre_statut, badge, senior, verified, faceToFace, webcam, responseTime, responseTimeDesc, key_admin,
    

In [ ]:
df = pd.DataFrame(master_list)

In [ ]:
df.to_csv('./data/superprof.csv', index=False)